In [ ]:
import json
import itertools
import networkx as nx
import pandas as pd
import random
from typing import Union, List, Dict
from collections import defaultdict

# --- FUNGSI PEMUATAN DAN PERSIAPAN DATA ---

def load_json_file(filename: str) -> Union[dict, list]:
    """Membuka dan memuat data dari sebuah file JSON."""
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File '{filename}' tidak ditemukan.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Gagal mem-parsing JSON dari file '{filename}'.")
        return None

def build_network_graph(network_data: dict) -> nx.Graph:
    """Membangun graf jaringan dari data networkDefinition.json."""
    G = nx.Graph()
    if not network_data: return G
    for entity in network_data.get('entity', []):
        G.add_node(entity['id'], **entity)
    for link in network_data.get('link', []):
        G.add_edge(link['s'], link['d'], PR=link['PR'], BW=link['BW'])
    return G

def build_placement_matrix(placement_data: dict) -> dict:
    """Membangun kamus penempatan dari data allocDefinition.json."""
    placement_matrix = defaultdict(list)
    if not placement_data: return {}
    for allocation in placement_data.get('initialAllocation', []):
        module_name = allocation["module_name"]
        resource_id = allocation["id_resource"]
        if resource_id not in placement_matrix[module_name]:
            placement_matrix[module_name].append(resource_id)
    return dict(placement_matrix)

# --- FUNGSI PERHITUNGAN DELAY (Sama seperti sebelumnya) ---

def _calculate_path_delay(path: list, graph: nx.Graph, packet_size: float) -> float:
    """Menghitung total delay di sepanjang sebuah path."""
    total_delay = 0
    for i in range(len(path) - 1):
        u, v = path[i], path[i+1]
        link_data = graph.get_edge_data(u, v)
        if link_data:
            bw_ms = link_data.get('BW', 1) / 1000.0
            pr_ms = link_data.get('PR', 0)
            transmission_delay = packet_size / bw_ms if bw_ms > 0 else float('inf')
            link_delay = pr_ms + transmission_delay
            total_delay += link_delay
    return total_delay

def calculate_adra(start_node: int, end_node: int, graph: nx.Graph, packet_size: float) -> float:
    """Menghitung ADRA: Delay dari requester ke titik masuk aplikasi."""
    try:
        path = nx.shortest_path(graph, source=start_node, target=end_node)
        return _calculate_path_delay(path, graph, packet_size)
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        return float('inf')

def calculate_adsa(target_app: dict, placement_matrix: dict, network_graph: nx.Graph, packet_size: float) -> float:
    """Menghitung ADSA: Rata-rata delay antar layanan dalam satu aplikasi."""
    service_list = [module['name'] for module in target_app.get('module', [])]
    all_device_locations = set()
    for service_name in service_list:
        if service_name in placement_matrix:
            all_device_locations.update(placement_matrix[service_name])
    if len(all_device_locations) < 2: return 0.0
    services_delay_list = []
    for dev1, dev2 in itertools.combinations(list(all_device_locations), 2):
        delay = calculate_adra(dev1, dev2, network_graph, packet_size)
        if delay != float('inf'):
            services_delay_list.append(delay)
    if not services_delay_list: return 0.0
    return sum(services_delay_list) / len(services_delay_list)

# --- BLOK EKSEKUSI UTAMA YANG DIMODIFIKASI ---
if __name__ == "__main__":
    # 1. Muat semua data dari file JSON (dengan path ke folder data)
    network_def = load_json_file('exp_rev/networkDefinition.json')
    app_def = load_json_file('exp_rev/appDefinition.json')
    alloc_def = load_json_file('exp_rev/allocDefinitionGA.json')

    if not all([network_def, app_def, alloc_def]):
        print("Satu atau lebih file definisi tidak dapat dimuat. Program berhenti.")
    else:
        # 2. Persiapkan struktur data
        network_graph = build_network_graph(network_def)
        placement_matrix = build_placement_matrix(alloc_def)
        
        # Siapkan daftar node untuk requester acak (kecuali node cloud ID 100)
        potential_requesters = [node['id'] for node in network_def.get('entity', []) if node.get('type') != 'CLOUD']
        if not potential_requesters:
            potential_requesters = [node for node in network_graph.nodes() if node != 100]

        # 3. Loop untuk semua aplikasi dan simpan hasilnya
        results_list = []
        PACKET_SIZE_BITS = 1_500_000  # Ukuran paket dalam bit (1.5 MB)

        for app_data in app_def:
            app_id = app_data['name']
            
            # Hitung ADSA
            adsa_result = calculate_adsa(app_data, placement_matrix, network_graph, PACKET_SIZE_BITS)
            
            # Hitung ADRA dengan requester acak
            entry_message = f"M.USER.APP.{app_id}"
            entry_module_name = next((t.get("module") for t in app_data.get("transmission", []) if t.get("message_in") == entry_message), None)
            
            adra_result = 0.0
            if entry_module_name and entry_module_name in placement_matrix and potential_requesters:
                entry_node = placement_matrix[entry_module_name][0]
                random_requester_id = random.choice(potential_requesters)
                adra_result = calculate_adra(random_requester_id, entry_node, network_graph, PACKET_SIZE_BITS)
            
            # Simpan hasil
            results_list.append({
                'app_id': app_id,
                'adra_ms': adra_result if adra_result != float('inf') else None,
                'adsa_ms': adsa_result,
                'total_delay_ms': (adra_result + adsa_result) if adra_result != float('inf') else None
            })

        # 4. Buat DataFrame dan lakukan analisis
        df_results = pd.DataFrame(results_list)
        df_results.set_index('app_id', inplace=True)

        # convert delay to seconds
        df_results['adra_s'] = df_results['adra_ms'] / 1000 if df_results['adra_ms'] is not None else None
        df_results['adsa_s'] = df_results['adsa_ms'] / 1000 if df_results['adsa_ms'] is not None else None
        df_results['total_delay_s'] = df_results['total_delay_ms'] / 1000 if df_results['total_delay_ms'] is not None else None
        
        print("--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---")
        print(df_results.to_string(float_format="%.2f"))
        print("\n" + "="*50 + "\n")

        print("--- Analisis Rata-rata Delay ---")
        
        # calculate average from adra_ms and adsa_ms
        average_adra = df_results['adra_ms'].mean()
        average_adra_s = average_adra / 1000 if average_adra is not None else None
        average_adsa = df_results['adsa_ms'].mean()
        average_adsa_s = average_adsa / 1000 if average_adsa is not None else None
        print(f"Rata-rata ADRA: {average_adra:.2f} ms ({average_adra_s:.2f} s)")
        print(f"Rata-rata ADSA: {average_adsa:.2f} ms ({average_adsa_s:.2f} s)")

        # Menghitung rata-rata keseluruhan dengan mengabaikan nilai None (inf)
        average_total_delay = df_results['total_delay_ms'].mean()
        print(f"Rata-rata Total Delay (ADRA + ADSA) untuk semua aplikasi: {average_total_delay:.2f} ms")
        
        # Aplikasi dengan delay terendah dan tertinggi
        app_min_delay = df_results['total_delay_ms'].idxmin()
        min_delay = df_results['total_delay_ms'].min()
        app_max_delay = df_results['total_delay_ms'].idxmax()
        max_delay = df_results['total_delay_ms'].max()
        
        print(f"Aplikasi dengan delay terendah: App '{app_min_delay}' ({min_delay:.2f} ms)")
        print(f"Aplikasi dengan delay tertinggi: App '{app_max_delay}' ({max_delay:.2f} ms)")


In [ ]:
if __name__ == "__main__":
    # 1. Muat semua data dari file JSON
    network_def = load_json_file('exp_rev/networkDefinition.json')
    app_def = load_json_file('exp_rev/appDefinition.json')
    alloc_def = load_json_file('exp_rev/allocDefinitionGA.json')

    if not all([network_def, app_def, alloc_def]):
        print("Satu atau lebih file definisi tidak dapat dimuat. Program berhenti.")
    else:
        # 2. Persiapkan struktur data
        network_graph = build_network_graph(network_def)
        placement_matrix = build_placement_matrix(alloc_def)
        
        # Siapkan daftar node untuk requester acak (kecuali node cloud ID 100)
        potential_requesters = [node['id'] for node in network_def.get('entity', []) if node.get('type') != 'CLOUD']
        if not potential_requesters:
            potential_requesters = [node for node in network_graph.nodes() if node != 100]

        # 3. Loop untuk semua aplikasi dan simpan hasilnya
        results_list = []
        PACKET_SIZE_BITS = 1_500_000  # Ukuran paket dalam bit (1.5 MB)

        for app_data in app_def:
            app_id = app_data['name']
            
            # Hitung ADSA
            adsa_result = calculate_adsa(app_data, placement_matrix, network_graph, PACKET_SIZE_BITS)
            
            # Hitung ADRA dengan requester acak
            entry_message = f"M.USER.APP.{app_id}"
            entry_module_name = next((t.get("module") for t in app_data.get("transmission", []) if t.get("message_in") == entry_message), None)
            
            adra_result = 0.0
            if entry_module_name and entry_module_name in placement_matrix and potential_requesters:
                entry_node = placement_matrix[entry_module_name][0]
                random_requester_id = random.choice(potential_requesters)
                adra_result = calculate_adra(random_requester_id, entry_node, network_graph, PACKET_SIZE_BITS)
            
            # Simpan hasil
            results_list.append({
                'app_id': app_id,
                'adra_ms': adra_result if adra_result != float('inf') else None,
                'adsa_ms': adsa_result,
                'total_delay_ms': (adra_result + adsa_result) if adra_result != float('inf') else None
            })

        # 4. Buat DataFrame dan lakukan analisis
        df_results = pd.DataFrame(results_list)
        df_results.set_index('app_id', inplace=True)

        # convert delay to seconds
        df_results['adra_s'] = df_results['adra_ms'] / 1000 if df_results['adra_ms'] is not None else None
        df_results['adsa_s'] = df_results['adsa_ms'] / 1000 if df_results['adsa_ms'] is not None else None
        df_results['total_delay_s'] = df_results['total_delay_ms'] / 1000 if df_results['total_delay_ms'] is not None else None
        
        print("--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---")
        print(df_results.to_string(float_format="%.2f"))
        print("\n" + "="*50 + "\n")

        print("--- Analisis Rata-rata Delay ---")
        
        # calculate average from adra_ms and adsa_ms
        average_adra = df_results['adra_ms'].mean()
        average_adra_s = average_adra / 1000 if average_adra is not None else None
        average_adsa = df_results['adsa_ms'].mean()
        average_adsa_s = average_adsa / 1000 if average_adsa is not None else None
        print(f"Rata-rata ADRA: {average_adra:.2f} ms ({average_adra_s:.2f} s)")
        print(f"Rata-rata ADSA: {average_adsa:.2f} ms ({average_adsa_s:.2f} s)")

        # Menghitung rata-rata keseluruhan dengan mengabaikan nilai None (inf)
        average_total_delay = df_results['total_delay_ms'].mean()
        print(f"Rata-rata Total Delay (ADRA + ADSA) untuk semua aplikasi: {average_total_delay:.2f} ms")
        
        # Aplikasi dengan delay terendah dan tertinggi
        app_min_delay = df_results['total_delay_ms'].idxmin()
        min_delay = df_results['total_delay_ms'].min()
        app_max_delay = df_results['total_delay_ms'].idxmax()
        max_delay = df_results['total_delay_ms'].max()
        
        print(f"Aplikasi dengan delay terendah: App '{app_min_delay}' ({min_delay:.2f} ms)")
        print(f"Aplikasi dengan delay tertinggi: App '{app_max_delay}' ({max_delay:.2f} ms)")

In [ ]:
# based on appDefinition.json, allocDefinition.json, networkDefinition.json
# give me list of client id and entry point for each application

def load_app_entry_points(app_def: list, placement_matrix: dict) -> dict:
    """Mengambil daftar ID klien dan titik masuk untuk setiap aplikasi."""
    app_entry_points = {}
    for app in app_def:
        app_id = app['name']
        entry_message = f"M.USER.APP.{app_id}"
        entry_module_name = next((t.get("module") for t in app.get("transmission", []) if t.get("message_in") == entry_message), None)
        
        if entry_module_name and entry_module_name in placement_matrix:
            # Get all entry points for this module
            entry_nodes = placement_matrix[entry_module_name]
            app_entry_points[app_id] = entry_nodes
    return app_entry_points

# Get entry points for all applications
app_entry_points = load_app_entry_points(app_def, placement_matrix)

# Print in the requested format
for app_id, entry_points_list in app_entry_points.items():
    print(f"app: {app_id}, entry_point: {entry_points_list}")

# Create a summary DataFrame for better visualization
entry_points_summary = []
for app_id, entry_points_list in app_entry_points.items():
    entry_points_summary.append({
        'app_id': app_id,
        'entry_points': entry_points_list,
        'num_entry_points': len(entry_points_list)
    })

summary_df = pd.DataFrame(entry_points_summary)
summary_df.set_index('app_id', inplace=True)

print("\n--- Summary Table: Application Entry Points ---")
print(summary_df.to_string())

# Show statistics
print(f"\nTotal applications: {len(app_entry_points)}")
all_entry_points = [ep for ep_list in app_entry_points.values() for ep in ep_list]
print(f"Total entry points across all apps: {len(all_entry_points)}")
print(f"Unique entry points: {len(set(all_entry_points))}")

In [ ]:
# Get unique entry point nodes across all applications
unique_entry_points = set()
for entry_points_list in app_entry_points.values():
    unique_entry_points.update(entry_points_list)

# Convert to sorted list for better readability
unique_entry_points_sorted = sorted(list(unique_entry_points))

print("--- Recap: Unique Entry Point Nodes ---")
print(f"Unique entry point nodes: {unique_entry_points_sorted}")
print(f"Total unique entry point nodes: {len(unique_entry_points_sorted)}")

# Create a frequency analysis to see which nodes are most commonly used as entry points
entry_point_frequency = {}
for entry_points_list in app_entry_points.values():
    for node in entry_points_list:
        entry_point_frequency[node] = entry_point_frequency.get(node, 0) + 1

# Sort by frequency (descending)
sorted_by_frequency = sorted(entry_point_frequency.items(), key=lambda x: x[1], reverse=True)

print("\n--- Entry Point Node Frequency Analysis ---")
print("Node ID | Frequency (Number of apps using this node)")
print("-" * 50)
for node_id, freq in sorted_by_frequency:
    print(f"{node_id:7} | {freq}")

# Identify cloud nodes vs edge nodes
cloud_nodes = [node for node in unique_entry_points_sorted if node == 100]  # Assuming 100 is cloud
edge_nodes = [node for node in unique_entry_points_sorted if node != 100]

print("\n--- Node Type Classification ---")
print(f"Cloud nodes serving as entry points: {cloud_nodes}")
print(f"Edge nodes serving as entry points: {edge_nodes}")
print(f"Number of edge nodes: {len(edge_nodes)}")
print(f"Number of cloud nodes: {len(cloud_nodes)}")